In [57]:
# Use the original_model_subbasins table of name and area to match to the parsed mapfile geojson subbasins by area, and assign name(s). There can be multiple subbasins that have the same area, so multiple names are alowed.
original_model_subbasins_table =  r"C:\Users\MBMcmanus\OneDrive - Garver\Documents\Work\Lawton\GIS\orig_subbasin_names_area.csv"
mapfile_subbasins_geojson = r"C:\Users\MBMcmanus\OneDrive - Garver\Documents\Work\Lawton\GIS\SquawCreek_Subbasins.geojson"
basin = "Squaw"


In [58]:
import geopandas as gpd

# open the mapfile subbasins geojson
mapfile_subbasins = gpd.read_file(mapfile_subbasins_geojson)
mapfile_subbasins.head()

,ID,area_sq_mi,AreaMatches,name,geometry
0,1,0.0134,"SQUAW A-3-f, SQUAW A-16-d, SQUAW B-2-north, SQ...",SQUAW A-3-f,"POLYGON ((-220073.061 1287339.548, -220085.971..."
1,2,0.0683,"E. CACHE TRIB A a, WRATTON TRIB 1 c, MISSION C...",SQUAW B-1-c,"POLYGON ((-219614.008 1288213.157, -219564.117..."
2,3,0.0193,"SQUAW A-14-a, SQUAW B-2-north, SQUAW B-2-south...",SQUAW B-1-d,"POLYGON ((-219489.304 1287946.851, -219433.41 ..."
3,4,0.0396,"SQUAW B-3-a, SQUAW A-7-b, SQUAW A-11-b, SQUAW ...",SQUAW B-1-e,"POLYGON ((-219035.76 1287942.727, -219028.068 ..."
4,5,0.0347,"SQUAW A-14, SQUAW A-14-i, SQUAW B-3-a, SQUAW A...",SQUAW A-14-i,"POLYGON ((-221844.247 1287526.25, -221843.898 ..."


In [59]:
# remove the AreaMatches and name column
mapfile_subbasins = mapfile_subbasins.drop(columns=['AreaMatches', 'name'])
mapfile_subbasins.head()

,ID,area_sq_mi,geometry
0,1,0.0134,"POLYGON ((-220073.061 1287339.548, -220085.971..."
1,2,0.0683,"POLYGON ((-219614.008 1288213.157, -219564.117..."
2,3,0.0193,"POLYGON ((-219489.304 1287946.851, -219433.41 ..."
3,4,0.0396,"POLYGON ((-219035.76 1287942.727, -219028.068 ..."
4,5,0.0347,"POLYGON ((-221844.247 1287526.25, -221843.898 ..."


In [60]:
# open the original model subbasins table
import pandas as pd
original_model_subbasins = pd.read_csv(original_model_subbasins_table)
# rename the columns to match the mapfile subbasins
original_model_subbasins.rename(columns={'Area (sqmi)': 'area_sq_mi', 'Subbasin': 'name'}, inplace=True)
original_model_subbasins.head()

,name,area_sq_mi,Basin
0,ADJ. ELLSWORTH 104,67.13,E_Cache
1,ADJ. LAWTONKA 114,23.15,E_Cache
2,BEEF CREEK 140,18.34,E_Cache
3,E. C TO H.E. BAILIEY,0.74,E_Cache
4,E. C TO MEDICINE CK,10.40,E_Cache


In [61]:
# drop rows that are not in the basin
original_model_subbasins = original_model_subbasins[original_model_subbasins['Basin'].str.contains(basin)]
original_model_subbasins.drop(columns=['Basin'], inplace=True)
original_model_subbasins

,name,area_sq_mi
58,SQUAW 1,0.46
59,SQUAW 10,1.28
60,SQUAW 11,1.94
61,SQUAW 12,0.29
62,SQUAW 13,0.27
...,...,...
131,SQUAW C-3-c,0.45
132,SQUAW C-3-d,0.11
133,SQUAW C-5-a,0.06
134,SQUAW C-5-b,0.29


In [62]:
# for each row in mapfile subbasins, find the next closest area in the original model subbasins table until the difference is greater than .01 difference in found closest areas, and assign the name(s) to the mapfile subbasin. If there are multiple matching areas within .01 sqmi, assign them as a list.
mapfile_subbasins['name'] = None
mapfile_subbasins['name'] = mapfile_subbasins['name'].astype(object) # make sure the name column is an object type to allow for lists
for index, row in mapfile_subbasins.iterrows():
    # get the area of the mapfile subbasin
    mapfile_area = row['area_sq_mi']
    # find the closest area in the original model subbasins table
    original_model_subbasins['area_diff'] = abs(original_model_subbasins['area_sq_mi'] - mapfile_area)
    # sort the original model subbasins by area_diff
    original_model_subbasins.sort_values('area_diff', inplace=True)
    # get the closest area and name(s) within .01 sqmi
    closest_areas = original_model_subbasins[original_model_subbasins['area_diff'] < .01]
    # if there are multiple closest areas, assign them as a list
    if len(closest_areas) > 0:
        mapfile_subbasins.at[index, 'name'] = list(closest_areas['name'])
    # if there is only one closest area, assign it as a string
    elif len(closest_areas) == 1:
        mapfile_subbasins.at[index, 'name'] = closest_areas['name'].values[0]
    # if there are no closest areas, assign None
    else:
        mapfile_subbasins.at[index, 'name'] = None
mapfile_subbasins


,ID,area_sq_mi,geometry,name
0,1,0.0134,"POLYGON ((-220073.061 1287339.548, -220085.971...","[SQUAW A-16-d, SQUAW A-3-f, SQUAW B-2-north, S..."
1,2,0.0683,"POLYGON ((-219614.008 1288213.157, -219564.117...","[SQUAW A-16-b, SQUAW A-9-a, SQUAW B-1-c, SQUAW..."
2,3,0.0193,"POLYGON ((-219489.304 1287946.851, -219433.41 ...","[SQUAW B-1-d, SQUAW B-2-south, SQUAW A-14-a, S..."
3,4,0.0396,"POLYGON ((-219035.76 1287942.727, -219028.068 ...","[SQUAW B-3-a, SQUAW B-1-e, SQUAW C-2, SQUAW A-..."
4,5,0.0347,"POLYGON ((-221844.247 1287526.25, -221843.898 ...","[SQUAW A-14, SQUAW A-14-i, SQUAW B-3-a, SQUAW ..."
...,...,...,...,...
89,132,1.2291,"POLYGON ((-217151.019 1285694.115, -217151.019...",None
90,151,0.2295,"POLYGON ((-217841.645 1282042.694, -217759.988...","[SQUAW 7, SQUAW 5]"
91,151,0.2295,"POLYGON ((-217841.645 1282042.694, -217842.377...","[SQUAW 7, SQUAW 5]"
92,167,0.0460,"POLYGON ((-221215.481 1286314.898, -221183.346...","[SQUAW A-16-e, SQUAW A-3-b, SQUAW B-1-a, SQUAW..."


In [63]:
# if there is only one name, assign it as a string, if there are multiple names, assign them as a list
mapfile_subbasins['name'] = mapfile_subbasins['name'].apply(lambda x: x[0] if isinstance(x, list) and len(x) == 1 else x)
mapfile_subbasins

,ID,area_sq_mi,geometry,name
0,1,0.0134,"POLYGON ((-220073.061 1287339.548, -220085.971...","[SQUAW A-16-d, SQUAW A-3-f, SQUAW B-2-north, S..."
1,2,0.0683,"POLYGON ((-219614.008 1288213.157, -219564.117...","[SQUAW A-16-b, SQUAW A-9-a, SQUAW B-1-c, SQUAW..."
2,3,0.0193,"POLYGON ((-219489.304 1287946.851, -219433.41 ...","[SQUAW B-1-d, SQUAW B-2-south, SQUAW A-14-a, S..."
3,4,0.0396,"POLYGON ((-219035.76 1287942.727, -219028.068 ...","[SQUAW B-3-a, SQUAW B-1-e, SQUAW C-2, SQUAW A-..."
4,5,0.0347,"POLYGON ((-221844.247 1287526.25, -221843.898 ...","[SQUAW A-14, SQUAW A-14-i, SQUAW B-3-a, SQUAW ..."
...,...,...,...,...
89,132,1.2291,"POLYGON ((-217151.019 1285694.115, -217151.019...",None
90,151,0.2295,"POLYGON ((-217841.645 1282042.694, -217759.988...","[SQUAW 7, SQUAW 5]"
91,151,0.2295,"POLYGON ((-217841.645 1282042.694, -217842.377...","[SQUAW 7, SQUAW 5]"
92,167,0.0460,"POLYGON ((-221215.481 1286314.898, -221183.346...","[SQUAW A-16-e, SQUAW A-3-b, SQUAW B-1-a, SQUAW..."


In [64]:
# export the mapfile subbasins with the assigned names to a new geojson file
output_geojson = r"C:\Users\MBMcmanus\OneDrive - Garver\Documents\Work\Lawton\GIS\SquawCreek_Subbasins_NamedByNearestAreas.geojson"
mapfile_subbasins.to_file(output_geojson, driver='GeoJSON')
